<div class="header">
  <h1 style="float: left;">Visualizing Temporal Networks</h1>
  <img width=300px src="https://www.uni-koblenz-landau.de/de/koblenz/logo.png" style="position: relative;"/>
</div>

In [1]:
################
# Dependencies #
################
# Note: Never import anything directly always import modules or packages, not single functions or classes,
# We should also check and clean up dependencies at some point.
import io
import time
import zipfile

import fileupload
import IPython.display as ipydisplay
import ipywidgets as widgets
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pylab
import seaborn as sns

import vtna.data_import
import vtna.graph
import vtna.layout
import vtna.utility

import main

In [2]:
#Install extentions:
#https://github.com/peteut/ipython-file-upload
#https://github.com/ipython-contrib/jupyter_contrib_nbextensions

help_input = """
<input onclick="show_help()" type="image" src="images/help.png" style='max-width: 20px;' alt="Submit Button" />
"""

helpscript = """
    <script> 
        function show_help(){
            alert("Please upload tab-separated values, see this example:");
        }
    </script>"""
display(ipydisplay.HTML(helpscript))

In [3]:
###############
# Data Import #
###############
# TODO: Would be great to have notes on where each layout is used.
bordered_box_layout = widgets.Layout(border='solid 1px rgb(210,210,210)', margin="5px 5px 5px 5px")
box_layout = widgets.Layout(margin="10px 0px 10px 0px")
simbox_layout = widgets.Layout(border='solid 1px rgb(210,210,210)',margin="5px 5px 5px 5px",width="100%",height="600px")
upload_layout = widgets.Layout(width="100%")



# Toggle button: Switch between Computer and Network upload
upload_type_toggle_button = widgets.ToggleButtons(
    options=['Computer', 'Network'],
    description='Source:',
    disabled=False,
    button_style='info',
    tooltips=['Upload file from local folder', 'Upload from network path'],
    layout=upload_layout
)
# Text input for Graph file path
graph_data_text = widgets.Text(
    value='',
    description='Graph File:',
    disabled=True,
    layout=upload_layout
)
# Text input for Metadata file path
metadata_text = widgets.Text(
    value='',
    description='Metadata File:',
    disabled=True, 
    layout=upload_layout
)

# Local upload for graph data
local_graph_file_upload = fileupload.FileUploadWidget(    
    label="Upload",
)
# Local upload for metadata
local_metadata_file_upload = fileupload.FileUploadWidget(    
    label="Upload",
)
# Network upload for graph data
network_graph_upload_button = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='info',
    tooltip='Upload graph file',
    icon='upload'
)
network_graph_upload_button.layout.display = 'none'
# Network upload for metadata
network_metadata_upload_button = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='info', 
    tooltip='Upload metadata',
    icon='upload'
)
network_metadata_upload_button.layout.display = 'none'
# Summary/Error output for graph data upload
graph_data_output = widgets.Output()
with graph_data_output:
    ipydisplay.display(ipydisplay.HTML(help_input))
# Summary/Error output for metadata upload
metadata_output = widgets.Output()
with metadata_output:
    ipydisplay.display(ipydisplay.HTML(help_input))
# Run button applies changes to metadata and open display view
run_button = widgets.Button(
    description='Run',
    disabled=False,
    button_style='success',
    tooltip='Run simulation',
    icon='start'
)
# Import menu button is shown in any not Import view and switches back to Import view.
import_menu_button = widgets.Button(
    description='Import',
    disabled=False,
    button_style='warning',
    tooltip='Import',
    icon='start'
)
w_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    description='Loading:',
    bar_style='info',
    orientation='horizontal'
)
w_progress.layout.visibility ='hidden'

# Box for toggle button
upload_type_hbox = widgets.HBox([upload_type_toggle_button], layout=box_layout)
# Box for graph data import
graph_data_upload_hbox = widgets.HBox([graph_data_text, local_graph_file_upload, network_graph_upload_button])
w_network_settings = widgets.HBox([graph_data_output])
import_graph_data_vbox = widgets.VBox([graph_data_upload_hbox, w_network_settings], layout=box_layout)
# Box for metadata import
metadata_upload_hbox = widgets.HBox([metadata_text, local_metadata_file_upload, network_metadata_upload_button])
metadata_configuration_left_vbox = widgets.VBox([])
metadata_configuration_hbox = widgets.HBox([metadata_configuration_left_vbox, metadata_output])
import_metadata_vbox = widgets.VBox([metadata_upload_hbox, metadata_configuration_hbox],layout=box_layout)
# Box for all import functionality
w_toolbar = widgets.HBox([run_button, w_progress])
full_import_vbox = widgets.VBox([upload_type_hbox, import_graph_data_vbox, import_metadata_vbox, w_toolbar])

# Create manager for import functionality
upload_manager = main.UIDataUploadManager(
                     local_graph_file_upload=local_graph_file_upload,
                     network_graph_upload_button=network_graph_upload_button,
                     graph_data_text=graph_data_text,
                     graph_data_output=graph_data_output,
                     local_metadata_file_upload=local_metadata_file_upload,
                     network_metadata_upload_button=network_metadata_upload_button,
                     metadata_text=metadata_text,
                     metadata_output=metadata_output,
                     metadata_configuration_vbox=metadata_configuration_left_vbox,
                     column_configuration_layout=box_layout
                )
# Attach event handlers to each interactable import widget
upload_type_toggle_button.observe(upload_manager.build_on_toggle_upload_type(), 'value')
local_graph_file_upload.observe(upload_manager.build_handle_local_upload_graph_data(), names='data')
local_metadata_file_upload.observe(upload_manager.build_handle_local_upload_metadata(), names='data')
network_graph_upload_button.on_click(upload_manager.build_handle_network_upload_graph_data())
network_metadata_upload_button.on_click(upload_manager.build_handle_network_upload_metadata())

In [4]:
#################
# Graph Display #
#################
# Note: Currently I just use time step, instead of time stamp, because it was easier to implement.
# Later we would replace it with timestamps, but this requires to map timestamp -> time step for proper
# functioning.
time_slider = widgets.IntSlider(
    description='Timeline: ',
    min=0,
    step=1,
    continuous_update=False
)

play = widgets.Play(
    value=0,
    min=0,
    step=1,
    description="Press play",
    disabled=False
)

# Create Display manager
display_manager = main.UIGraphDisplayManager(time_slider=time_slider, play=play)

In [5]:
###########################################
# Menu: Toggle between Import and Display #
###########################################
# Toggle between import tool and animation tool?
simulation_box = None



# Note: We should not build and initialize the simulation box before we have the data.
#     Because without the data a number of functions cannot be properly initiliazed.
#     The global simulation_box is a workaround, it should be replaced by some manager class.
#     It is possible that we should rename the display manager to graph manager,
#     and create a display manager responsible for the simulation box.
def on_run(b):
    # on_run will initialize the simulation_box, which contains the graph display.
    # it will initialize the graph object using the display_manager
    global simulation_box
    # Load imported imported graph data and metadata into the temporal graph
    display_manager.init_temporal_graph(
        edge_list=upload_manager.get_edge_list(),
        metadata=upload_manager.get_metadata(),
        granularity=600
    )
    plot = widgets.interactive(display_manager.build_display_current_graph(fig_num=1), 
                           current_time_step=time_slider)
    widgets.jslink((play, 'value'), (time_slider, 'value'))
    playbox = widgets.HBox([play, time_slider])
    graphbox = widgets.VBox([plot, playbox])
    simulation_box = widgets.VBox([import_menu_button,  graphbox], layout=simbox_layout)
    display(simulation_box)
    
    w_progress.layout.visibility ='visible'
    full_import_vbox.layout.display = 'none'
    simulation_box.layout.display = 'block'
    
def on_import(b):
    global simulation_box
    simulation_box.layout.display = 'none'
    w_progress.value=0
    w_progress.layout.visibility ='hidden'
    full_import_vbox.layout.display = 'block'
        
run_button.on_click(on_run)
import_menu_button.on_click(on_import)

# Show only Import Box
display(full_import_vbox)

A Jupyter Widget

In [6]:
display(ipydisplay.HTML('''
<script>
    code_show=true; 
    function code_toggle() {
         if (code_show){
             $('div.input').hide();
         } else {
             $('div.input').show();
         }
         code_show = !code_show
    } 
    $( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.
'''))